In [44]:
import pandas as pd
import sqlite3


In [45]:
con = sqlite3.connect('rusa.db')

In [46]:
pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'", con)

,name
0,riders
1,routes
2,event_details
3,events
4,event_results
5,clubs


In [4]:
q = \
"""
select * 
from
    event_results er
    join event_details ed
        on er.eid = ed.eid
limit 10"""
pd.read_sql(q, con)

,cert,rusa,eid,hours,minutes,medal,finish_minutes,eid,finishers,dnf,starters
0,373131,8037,5426,9,44,None,584,5426,29,0.0,29.0
1,373132,6328,5426,9,20,Y,560,5426,29,0.0,29.0
2,373133,5776,5426,11,20,Y,680,5426,29,0.0,29.0
3,373134,6151,5426,8,47,None,527,5426,29,0.0,29.0
4,373135,8313,5426,8,27,None,507,5426,29,0.0,29.0
5,373136,8334,5426,8,0,None,480,5426,29,0.0,29.0
6,373137,7372,5426,10,28,None,628,5426,29,0.0,29.0
7,373138,7,5426,11,45,Y,705,5426,29,0.0,29.0
8,373139,5909,5426,11,20,Y,680,5426,29,0.0,29.0
9,373140,6578,5426,9,35,Y,575,5426,29,0.0,29.0


### Compare me with another rider

In [5]:
RUSA_A = 11548
RUSA_B = 11549

In [33]:
MEGA_TABLE_SQL = \
"""
select
    e.date,
    c.club_name,
    rt.route,
    rt.distance,
    rd.rusa,
    rd.name,
    er.cert,
    er.hours, er.minutes
from
    events e
    join routes rt
        on e.rtid = rt.rtid
    join clubs c
        on e.host_acp_code = c.acp_code
    join event_results er
        on e.eid = er.eid
    join event_details ed
        on er.eid = ed.eid
    join riders rd
        on er.rusa = rd.rusa"""

In [40]:
q = \
f"""
with 
    mt as ({MEGA_TABLE_SQL}),
    rusa_a as (
        select *
        from mt
        where mt.rusa = {RUSA_A}
    ),
    rusa_b as (
        select *
        from mt
        where mt.rusa = {RUSA_B}
    )

select
    mt.cert, mt.date, mt.club_name, mt.distance, mt.route,
    rusa_a.rusa rusa_a, rusa_a.name name_a,
    rusa_b.rusa rusa_b, rusa_b.name name_b
from
    mt
    left join rusa_a on mt.cert = rusa_a.cert
    left join rusa_b on mt.cert = rusa_b.cert
where
    rusa_a.rusa is not null or
    rusa_b.rusa is not null
"""
pd.read_sql(q, con)

,cert,date,club_name,distance,route,rusa_a,name_a,rusa_b,name_b
0,116899,2011/02/05,Pacific Coast Highway Randonneurs,300.0,OC Circle Santiago Peak,NaN,None,11549.0,"PRATT, Chris"
1,116899,2011/02/05,Pacific Coast Highway Randonneurs,300.0,OC Circle Santiago Peak,NaN,None,11549.0,"PRATT, Chris"
2,127518,2012/05/05,Pennsylvania Randonneurs,300.0,Water Gap 300 (C),NaN,None,11549.0,"PRATT, Chris"
3,127552,2012/05/12,New Jersey Randonneurs,300.0,route 15 (unnamed),NaN,None,11549.0,"PRATT, Chris"
4,94461,2013/08/24,Seattle International Randonneurs,400.0,SIR 2013 Summer 400k,NaN,None,11549.0,"PRATT, Chris"
...,...,...,...,...,...,...,...,...,...
277,RUSA-P16852,2020/02/22,Houston Randonneurs,100.0,Sunnyside Loop,NaN,None,11549.0,"PRATT, Chris"
278,RUSA-P16852,2020/02/22,Houston Randonneurs,100.0,Sunnyside Loop,NaN,None,11549.0,"PRATT, Chris"
279,RUSA-P16852,2020/02/22,Houston Randonneurs,111.0,Sunnyside Loop,NaN,None,11549.0,"PRATT, Chris"
280,RUSA-P16852,2020/02/22,Houston Randonneurs,111.0,Sunnyside Loop,NaN,None,11549.0,"PRATT, Chris"


In [57]:
q = \
f"""
select *
    
from 
    event_results er
    join events e on er.eid = e.eid    
    join routes rt on e.rtid = rt.rtid

where 
    e.type <> 'RUSAB'
    and er.rusa in ({RUSA_A}, {RUSA_B})
order by e.date
limit 20
"""
pd.read_sql(q, con)

,cert,rusa,eid,hours,minutes,medal,finish_minutes,eid,host_acp_code,date,rtid,type,rtid,route,distance
0,543970,11549,8388,9,45,None,585,8388,943030,2017/01/07,423.0,ACPB,423.0,route 423 (unnamed),200.0
1,543970,11549,8388,9,45,None,585,8388,943030,2017/01/07,423.0,ACPB,423.0,route 423 (unnamed),202.0
2,544038,11549,8350,11,15,None,675,8350,943025,2017/01/14,1722.0,ACPB,1722.0,No Return to Fitzhugh 200K,200.0
3,544038,11549,8350,11,15,None,675,8350,943025,2017/01/14,1722.0,ACPB,1722.0,No Return to Fitzhugh 200K,206.0
4,191216,11549,8391,15,30,None,930,8391,943030,2017/01/28,87.0,ACPB,87.0,route 87 (unnamed),300.0
5,191278,11549,8351,18,19,None,1099,8351,943025,2017/02/04,1726.0,ACPB,1726.0,Mueller Double Century,300.0
6,191278,11549,8351,18,19,None,1099,8351,943025,2017/02/04,1726.0,ACPB,1726.0,Mueller Double Century,324.0
7,124709,11549,8394,20,53,None,1253,8394,943030,2017/02/11,1064.0,ACPB,1064.0,Brookshire-Columbus 400K,400.0
8,124709,11549,8394,20,53,None,1253,8394,943030,2017/02/11,1064.0,ACPB,1064.0,Brookshire-Columbus 400K,407.0
9,195416,11549,8354,16,13,None,973,8354,943025,2017/02/25,1852.0,ACPB,1852.0,Pedernales DC 2,300.0


In [67]:
## uh oh, our routes tables is messed up
q = \
f"""
with dupe_routes as (
select rtid, count(*) dupe_rows
from routes
group by 1 order by 2 desc
)
select *
from 
    routes rt
    join dupe_routes r2 on rt.rtid = r2.rtid
where dupe_rows > 3
order by dupe_rows desc, rtid, distance
"""
pd.read_sql(q, con)

,rtid,route,distance,rtid,dupe_rows
0,-1.0,route TBD (unnamed),100.0,-1.0,9
1,-1.0,route TBD (unnamed),110.0,-1.0,9
2,-1.0,route TBD (unnamed),120.0,-1.0,9
3,-1.0,route TBD (unnamed),200.0,-1.0,9
4,-1.0,route TBD (unnamed),300.0,-1.0,9
5,-1.0,route TBD (unnamed),360.0,-1.0,9
6,-1.0,route TBD (unnamed),400.0,-1.0,9
7,-1.0,route TBD (unnamed),600.0,-1.0,9
8,-1.0,route TBD (unnamed),1000.0,-1.0,9
9,1565.0,Escape to Wisconsin 2022,100.0,1565.0,6
